In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

/Users/rohitramaswamy/Desktop/programming-stuff/sports-data-scraping/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [35]:
# Define the function to separate name and code
def separate_name_and_code(input_string):
    name, code = input_string.split(' (')
    code = code.rstrip(')')
    return pd.Series([name, code])


def clean_countries(text):
    text = text.replace('ICC', '')
    text = text.replace('Afr', '')
    text = text.replace('/', '')
    text = text.replace('Asia', '')

    return text
# Initialize current_df correctly
current_df = pd.DataFrame(columns=["Player"])

year = 1980
for i in range(2025-year):
    url = f'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;spanmax1=10+Jul+{year};spanval1=span;orderby=sixes;template=results;type=batting'
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check if the request was successful

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all tables in the page
        tables = soup.find_all('table')

        # Convert the first table to a pandas DataFrame
        if tables:
            df = pd.DataFrame(pd.read_html(StringIO(str(tables[2])))[0]).head(20)
            if 'Player' in df.columns and '6s' in df.columns:
                df = df[['Player', '6s']]
                df.rename(columns={'6s': f'{year}'}, inplace=True)
                df[['Player', 'Country']] = df['Player'].apply(separate_name_and_code)

                current_df = pd.merge(current_df, df, on='Player', how='outer')
                current_df['Country'] = current_df['Country_y'].combine_first(current_df['Country_x'])
                current_df = current_df.drop(columns=['Country_x', 'Country_y'])




            else:
                print(f"'Player' or 'Runs' column not found for the year {year}")
        else:
            print("No tables found on the page.")
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e.response.status_code} - {e.response.reason}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    year = year + 1
    





# Apply the function and create two new columns


# current_df['Country'] = current_df['Country'].apply(clean_countries)
# Apply the function to the column


print(current_df)


An error occurred: 'Country_y'
            Player 1980 1981 1982 1983 1984 1985 1986 1987 1988  ... 2016  \
0       A Flintoff  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
1         A Jadeja  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
2      A Ranatunga  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
3        A Symonds  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
4   AB de Villiers  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  187   
..             ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
79     Wasim Akram  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
80      Wasim Raja   5+   5+   7+   9+  11+  NaN  NaN  NaN  NaN  ...  NaN   
81  Yashpal Sharma  NaN  NaN  NaN  10+  10+  NaN  NaN  NaN  NaN  ...  NaN   
82    Yuvraj Singh  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  148   
83    Zaheer Abbas   4+  NaN   8+  19+  21+  22+  22+  22+  22+  ...  NaN   

   2017 2018 2019 2020 2021 2022 2023 2024  

In [36]:

current_df['Country'] = current_df['Country'].apply(clean_countries)
# Apply the function to the column

current_df.replace(to_replace=r'\+', value='', regex=True, inplace=True)


In [37]:

# List of countries
countries = pd.read_csv(r'tools/shortcodes.csv', dtype=str)  

print(countries.tail(5))

# row_index = countries[countries["Code"] == 'IND'].index[0]
# print(row_index)

# Function to get Shortcode based on Code
def get_row(input):
    rows = countries[countries["Code"] == input].index
    if len(rows) > 0:
        row_index = rows[0]
        if len(countries.loc[row_index, "Lowercase shortcode"]) == 2:
            return f'https://public.flourish.studio/country-flags/svg/{countries.loc[row_index, "Lowercase shortcode"]}.svg'
        else:
            return countries.loc[row_index, "Lowercase shortcode"]
    else:
        return 'Unknown'  # Or any default value you want to assign

# Apply function to get Shortcode for each country in current_df
current_df["Flag"] = current_df["Country"].apply(get_row)

print(current_df.head(5))

            Name  Code Shortcode  \
245        Yemen    YE      :YE:   
246       Zambia    ZM      :ZM:   
247     Zimbabwe   ZIM     :ZIM:   
248  West Indies    WI       NaN   
249     Scotland  SCOT       NaN   

                                   Lowercase shortcode  
245                                                 ye  
246                                                 zm  
247                                                 zw  
248  https://upload.wikimedia.org/wikipedia/en/9/9b...  
249  https://upload.wikimedia.org/wikipedia/commons...  
           Player 1980 1981 1982 1983 1984 1985 1986 1987 1988  ... 2017 2018  \
0      A Flintoff  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
1        A Jadeja  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2     A Ranatunga  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
3       A Symonds  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4  AB de Villiers  NaN  NaN  NaN  Na

In [38]:
excel_file = "output.xlsx" 
 
# Write DataFrame to Excel 
current_df.to_excel(excel_file, index=False) 